In [6]:
import pandas as pd


class Coordinate:
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat

    def get_lon(self):
        return self.lon
    
    def get_lat(self):
        return self.lat

In [15]:
df_hdb = pd.read_csv('../data/modified/hdb_coordinates.csv')
df_hdb

,Unnamed: 0,address,latitude,longitude
0,0,174 ANG MO KIO AVE 4,1.375097,103.837619
1,1,541 ANG MO KIO AVE 10,1.373922,103.855621
2,2,163 ANG MO KIO AVE 4,1.373549,103.838176
3,3,446 ANG MO KIO AVE 10,1.367761,103.855357
4,4,557 ANG MO KIO AVE 10,1.371626,103.857736
...,...,...,...,...
7916,7916,634 WOODLANDS RING RD,1.436266,103.802943
7917,7917,810 WOODLANDS ST 81,1.442668,103.788139
7918,7918,844 YISHUN ST 81,1.415625,103.835447
7919,7919,654 YISHUN AVE 4,1.423053,103.840410


In [16]:
import os
import requests

from dotenv import load_dotenv


load_dotenv()
API_KEY_2 = os.getenv("OPEN_ROUTE_SERVICE_API_KEY")


def get_distance_between_two_coordinates(start: Coordinate, end: Coordinate) -> float:
    # Create the URL for the Directions API
    try:
        url = 'https://api.openrouteservice.org/v2/directions/driving-car'

        params = {
            'api_key': API_KEY_2,
            'start': f"{start.get_lon()},{start.get_lat()}",
            'end': f"{end.get_lon()},{end.get_lat()}"
        }

        # Make the request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract the distance from the response (in meters)
            distance = data['features'][0]['properties']['segments'][0]['distance']

            # convert distance to kilometers
            distance = distance / 1000
        else:
            distance = -1
        return distance
    except Exception:
        return -2

### Calculate distances for POI
Purpose: To calculate distances from addresses to POI
- distance2cbd
- distance2nearestschool
- distance2nearestmrt

How: Use OpenRouteService API to get distances

In [ ]:
CENTRAL_BUSINESS_DISTRICT = Coordinate(lon=103.851784, lat=1.287953)

address_list = df_hdb['address'].tolist()
latitude_list = df_hdb['latitude'].tolist()
longitude_list = df_hdb['longitude'].tolist()
distance_list = []

for i, address in enumerate(address_list):
    hdb_coordinate = Coordinate(lat=latitude_list[i], lon=longitude_list[0])
    distance = get_distance_between_two_coordinates(hdb_coordinate, CENTRAL_BUSINESS_DISTRICT)

    distance_list.append(distance)

distance_list

In [ ]:
df_hdb_distance2cbd = pd.DataFrame({
    'address': address_list, 
    'latitude': latitude_list, 
    'longitude': longitude_list,
    'distance2cbd': distance_list
})
